# MIL - Multi instance Learning

In [1]:
import os
import datetime
import copy
import re
import yaml
import uuid
import warnings
import time
import inspect

import numpy as np
import pandas as pd
from functools import partial, reduce
from random import shuffle
import random

import torch
from torch import nn, optim
from torch import nn
from torch.nn import functional as F
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torchvision.models import resnet
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn import metrics as mtx
from sklearn import model_selection as ms
import os
import shutil
import warnings
import torch
import torch.nn as nn
import torchvision.models as models
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch.nn.init as init
from PIL import Image
from datetime import datetime
import random


warnings.filterwarnings("ignore")

/tmp/ipykernel_3859/1677006143.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Dataset definition

In [2]:
# The data is not standardized because it has two different format
def calculate_age(dob_str):
    formats = ["%m/%d/%Y", "%d-%m-%Y"]  # Two different format at the same time!
    for format_string in formats:
        try:
            dob_date = datetime.strptime(dob_str, format_string)
            current_date = datetime.now()
            age = (
                current_date.year
                - dob_date.year
                - (
                    (current_date.month, current_date.day)
                    < (dob_date.month, dob_date.day)
                )
            )
            return age
        except ValueError:
            continue


# The data for the gender is not consistent F or f or M or m
def encode_gender(gender):
    if gender == "F" or gender == "f":  # Two different gender value F and f ....
        return 0
    elif gender == "M" or gender == "m":
        return 1
    else:
        raise ValueError(
            "Invalid gender value. Expected 'F' or 'M', but received: {}".format(gender)
        )


# The dataset for the multi instance training - Every patient has a bag of images
class DLMICustomDataset(Dataset):
    def __init__(self, data, transform=None, flag="trainset", max_images=80):
        self.data = data
        self.transform = transform
        self.flag = flag
        self.max_images = max_images

    def __len__(self):
        return len(self.data)

    # Get items by the index
    def __getitem__(self, idx):
        patient_ID = self.data.iloc[idx, 1]  # Get patient ID
        img_path_folder = "data/raw/" + self.flag + "/" + str(patient_ID) + "/"

        images = []
        images_loaded = 0
        # Load images and make sure that the loaded images bag has the same number of images
        for filename in os.listdir(img_path_folder):
            if filename.endswith(".jpg"):
                img_path = os.path.join(img_path_folder, filename)
                image = Image.open(img_path).convert("RGB")
                if self.transform:
                    image = self.transform(image)
                images.append(image)
                images_loaded += 1
                if (
                    images_loaded >= self.max_images
                ):  # Stop loading if max_images reached
                    break
        if len(images) == 0:
            images.append(torch.zeros((3, 224, 224)))  # Placeholder image

        # If not enough images founded, randomly pick image from the loaded images
        while len(images) < self.max_images:
            random_image = random.choice(images)
            images.append(random_image)

        label = torch.tensor(self.data.iloc[idx, 2])
        gender = torch.tensor(encode_gender(self.data.iloc[idx, 3]), dtype=torch.long)
        age = torch.tensor(calculate_age(self.data.iloc[idx, 4]), dtype=torch.float32)
        lymph_count = torch.tensor(self.data.iloc[idx, 5], dtype=torch.float32)
        clinical_data = torch.stack((gender, age, lymph_count))
        # Final size is (num_images, channels, height, width)
        return torch.stack(images), clinical_data, label

Model definition

In [3]:
class MILModel(nn.Module):
    def __init__(self, mlp_input_dim, mlp_hidden_dim):
        super(MILModel, self).__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        for param in self.resnet18.parameters():
            param.requires_grad = False
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(num_ftrs, num_ftrs)  # One trainable layers
        self.linear_classifier = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
        )  # One classifier with bias
        self.mlp = MLP(input_dim=mlp_input_dim, hidden_dim=mlp_hidden_dim, output_dim=1)
        self.activation = nn.Sigmoid()

    # Batch imaging forwarding for bags batchsize*imagesnum*channel*width * height
    def forward_images(self, image_data):
        score_output = []
        for batch in image_data:
            batch_encoded_images = []
            for image in batch:
                image = image.unsqueeze(0)
                features = self.resnet18(image)
                # Mean pooling to do max pooling or LSE pooling
                batch_encoded_images.append(features)
            feature_extracted = torch.stack(batch_encoded_images)
            features = torch.mean(feature_extracted, dim=0)  # mean pooling
            score_output.append(self.linear_classifier(features).view(-1))
        return torch.stack(score_output)

    def forward(self, image_data, clinical_data):
        batch_size, num_images, channels, height, width = image_data.size()
        image_output = self.forward_images(image_data)
        mlp_output = self.mlp(clinical_data)
        return image_output, mlp_output

    def loss_function(self, image_output, clinical_output, labels):
        image_loss = F.binary_cross_entropy_with_logits(
            image_output, labels
        )  # loss should be calculated before sigmoid for stable trainging
        clinical_loss = F.binary_cross_entropy_with_logits(
            clinical_output, labels
        )  # loss should be calculated before sigmoid for stable trainging

        return (image_loss + clinical_loss) / 2


class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.LeakyReLU()
        self.activate = nn.Sigmoid()

        # Initialize the weights for the linear layers
        init.xavier_uniform_(self.fc1.weight)
        init.xavier_uniform_(self.fc2.weight)
        init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Training

In [4]:
def calculate_accuracy(outputs, labels):
    predicted_classes = torch.argmax(outputs, dim=1)
    true_classes = torch.argmax(labels, dim=1)
    accuracy = (predicted_classes == true_classes).float().mean().item()
    return accuracy


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

Training parameters

In [5]:
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
import json

# from stanfordnlp.training import trainset_splitter


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Trianing is on " + str(device))
# Define hyperparameters
hyperparameters = {
    "mlp_input_dim": 3,
    "mlp_hidden_dim": 10,
    "learning_rate": 0.001,
    "batch_size": 12,
    "num_epochs": 500,
}

model = MILModel(
    mlp_input_dim=hyperparameters["mlp_input_dim"],
    mlp_hidden_dim=hyperparameters["mlp_hidden_dim"],
).to(device)

with open(f"hyperparameters.json", "w") as json_file:
    json.dump(hyperparameters, json_file)

current_time = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
log_dir = os.path.join("logs", current_time)
writer = SummaryWriter(log_dir)

data = pd.read_csv(
    "/home/lujun/local/DLMI-Classification/data/raw/clinical_annotation.csv"
)
train_data = data[data["LABEL"] != -1]
test_data = data[data["LABEL"] == -1]
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# training : validation : test = 130 : 33 : 42
training_dataset = DLMICustomDataset(
    data=train_data, transform=transform, flag="trainset"
)  # 130

test_dataset = DLMICustomDataset(
    data=test_data, transform=transform, flag="testset"
)  # 42

train_size = int(0.8 * len(training_dataset))
val_size = len(training_dataset) - train_size
train_dataset, val_dataset = random_split(training_dataset, [train_size, val_size])
train_loader = DataLoader(
    train_dataset, batch_size=hyperparameters["batch_size"], shuffle=True
)  # 130
val_loader = DataLoader(
    val_dataset, batch_size=hyperparameters["batch_size"], shuffle=False
)  # 33
test_loader = DataLoader(
    test_dataset, batch_size=hyperparameters["batch_size"], shuffle=False
)
# Every item: max-images * channel number * width * height

Trianing is on cuda


Start Training

In [17]:
optimizer = optim.Adam(model.parameters(), lr=hyperparameters["learning_rate"])
num_epochs = hyperparameters["num_epochs"]
model = model.to(device)
best_loss = float("inf")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for inputs, clinical, labels in train_loader:
        inputs, clinical, labels = (
            inputs.to(device),
            clinical.to(device),
            labels.to(device),
        )
        optimizer.zero_grad()
        image_output, mlp_output = model(inputs, clinical)
        labels = labels.view(image_output.shape).float()
        loss = model.loss_function(image_output, mlp_output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        combined_output = (
            torch.sigmoid(image_output) + torch.sigmoid(mlp_output)
        ) / 2  # After the calculation of loss, predict the label using activation function
        predicted_labels = torch.round(combined_output)
        correct = (predicted_labels == labels).sum().item()
        total_correct += correct
        total_samples += labels.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    accuracy = total_correct / total_samples
    print(
        "Epoch {} Total Loss: {:.4f}, Accuracy: {:.2f}%".format(
            epoch + 1, running_loss / len(train_loader), accuracy * 100
        )
    )
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Accuracy/train", accuracy, epoch)
    # Validation
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, clinical, labels in val_loader:
            inputs, clinical, labels = (
                inputs.to(device),
                clinical.to(device),
                labels.to(device),
            )
            image_output, mlp_output = model(inputs, clinical)
            labels = labels.view(image_output.shape).float()
            loss = model.loss_function(image_output, mlp_output, labels)
            val_loss += loss.item() * inputs.size(0)
            combined_output = (
                torch.sigmoid(image_output) + torch.sigmoid(mlp_output)
            ) / 2
            predicted_labels = torch.round(combined_output)
            correct += (predicted_labels == labels).sum().item()
            total += labels.size(0)
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    writer.add_scalar("Loss/validation", val_loss, epoch)
    writer.add_scalar("Accuracy/validation", val_accuracy, epoch)
    print(
        f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}"
    )

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        unchanged_count = 0
    else:
        unchanged_count += 1

    if unchanged_count >= 5:
        print("Loss has not changed for 5 consecutive epochs. Stopping training.")
        break

    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": epoch_loss,
            "hyperparameters": hyperparameters,
        },
        f"{log_dir}/trained_model_epoch_{epoch+1}.pth",
    )

Epoch [1/500], Loss: 1.1171
Epoch 1 Total Loss: 13.2016, Accuracy: 50.77%
Epoch [1/500], Train Loss: 1.1171, Val Loss: 1.2485, Val Accuracy: 0.4848
Epoch [2/500], Loss: 1.1015
Epoch 2 Total Loss: 13.0172, Accuracy: 51.54%
Epoch [2/500], Train Loss: 1.1015, Val Loss: 1.2334, Val Accuracy: 0.4848
Epoch [3/500], Loss: 1.0889
Epoch 3 Total Loss: 12.8684, Accuracy: 51.54%
Epoch [3/500], Train Loss: 1.0889, Val Loss: 1.2142, Val Accuracy: 0.4848
Epoch [4/500], Loss: 1.0768
Epoch 4 Total Loss: 12.7256, Accuracy: 51.54%
Epoch [4/500], Train Loss: 1.0768, Val Loss: 1.2046, Val Accuracy: 0.4848
Epoch [5/500], Loss: 1.0666
Epoch 5 Total Loss: 12.6057, Accuracy: 52.31%
Epoch [5/500], Train Loss: 1.0666, Val Loss: 1.1926, Val Accuracy: 0.4545
Epoch [6/500], Loss: 1.0564
Epoch 6 Total Loss: 12.4846, Accuracy: 52.31%
Epoch [6/500], Train Loss: 1.0564, Val Loss: 1.1825, Val Accuracy: 0.4545
Epoch [7/500], Loss: 1.0477
Epoch 7 Total Loss: 12.3822, Accuracy: 52.31%
Epoch [7/500], Train Loss: 1.0477, Val

KeyboardInterrupt: 

Model evaluation

In [34]:
checkpoint = torch.load(f"logs/2024-03-27_23-20-25/trained_model_epoch_100.pth")
model.load_state_dict(checkpoint["model_state_dict"])

# test
model.eval()  # Set model to evaluation mode
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    predicted_labels_list = []
    for inputs, clinical, labels in train_loader:
        inputs, clinical, labels = (
            inputs.to(device),
            clinical.to(device),
            labels.to(device),
        )
        image_output, mlp_output = model(inputs, clinical)
        labels = labels.view(image_output.shape).float()
        loss = model.loss_function(image_output, mlp_output, labels)
        val_loss += loss.item() * inputs.size(0)
        combined_output = (torch.sigmoid(image_output) + torch.sigmoid(mlp_output)) / 2
        predicted_labels = torch.round(combined_output)
        predicted_labels_list.append(predicted_labels)

In [50]:
from sklearn.metrics import balanced_accuracy_score

# Concatenate predicted labels from all batches
predicted_labels_tensor = torch.cat(predicted_labels_list)
# Convert predicted labels tensor to numpy array
predicted_labels_np = np.squeeze(predicted_labels_tensor.cpu().numpy())
# Convert true labels tensor to numpy array
true_labels_np = torch.cat([labels for _, _, labels in train_loader]).cpu().numpy()
# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(true_labels_np, predicted_labels_np)
print(f"Balanced Accuracy: {balanced_acc}")

Balanced Accuracy: 0.47916666666666663


In [35]:
combined_tensor = torch.cat(predicted_labels_list, dim=0)
combined_tensor.shape
combined_array = combined_tensor.cpu().numpy()
ID = test_dataset.data.ID.to_numpy()
ID = ID[:, np.newaxis]
result_array = np.concatenate((ID, combined_array), axis=1)
result_df = pd.DataFrame(result_array)
result_df.columns = ["ID", "Predicted"]
result_df["Predicted"] = result_df["Predicted"].astype(int)
result_df.to_csv("output4.csv", index=False)